In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ ['KERAS_BACKEND']='theano' 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

Shape of dataset  (9693, 2)
Index(['message', 'class'], dtype='object')
No. of unique classes 4


,message,class
0,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
1,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
2,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
3,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL
4,#انتخبوا_العرص #انتخبوا_البرص #مرسى_رئيسى #اين...,NEUTRAL
5,امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #...,POS
6,أحكي لكم قصة حدثت بين شاب و أحد الفتيات كان بي...,OBJ
7,أدعوكم لحضور الندوة الثقافية الأربعاء مركز اعد...,OBJ
8,عودة جماعة الإخوان إلى الحياة السياسية بنفس وض...,NEG
9,_ علاج السكري #اف_نتشورال #السعودية #العراق #ا...,OBJ


In [6]:
categories = {
    'iman':1, 
    'shalat':5, 
    'zakat':2, 
    'puasa':4, 
    'haji':3, 
}
arabicCategories = {
    'iman':'الايمن', 
    'shalat':'الصَّلَاةِ', 
    'zakat':'الزَّكَاةِ ', 
    'puasa':'الصَوْمِ ', 
    'haji':'الحج', 
}

In [13]:
testingArticles = [
    'algeria.txt',
    'algeria2.txt',
    'entertaiment2.txt',
    'entertainment.txt',
    'religion.txt',
    'religion2.txt',
    'society.txt',
    'society2.txt',
    'sport.txt',
    'sport2.txt',
    'world.txt',
    'world2.txt'
]
articlePath = "Articles/"+testingArticles[1]
#article = open(articlePath, 'r').read()